<a href="https://colab.research.google.com/github/yutawatabe/pyCGE/blob/main/EK_Exacthatalgebra_ENG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculate Counterfactual Equilibrim

In [1]:
# Import numpy and math
import numpy as np # numpy for matrix calcualtion
from math import gamma # math.gamma for gamma function calculation

# Start by resolving the equilbrium with different parameters.

Simulate parameters and calculate the EK equilibrium.

In [5]:
def updatewage(w, theta, sigma, N, L_S, T, d, psi):
    """
    Update wages based on parameters and economic variables.
    This function calculates new wage values based on input parameters and economic variables.

    Parameters:
    w (np.ndarray): Array of current wage rates for different countries.
    theta (float): Elasticity of trade.
    sigma (float): Elasticity of substitution for goods.
    N (int): Number of countries.
    L_S (np.ndarray): Labor supply for each country.
    T (np.ndarray): Technology level for each country.
    d (np.ndarray): Trade costs between countries.
    psi (float): Wage adjustment parameter.

    Returns:
    w_new (np.ndarray): Updated wages for countries.
    Z (np.ndarray): Excess labor demand for each country.
    P (np.ndarray): Price indices.
    X (np.ndarray): Trade flows between countries.
    """

    Xn = w * L

    ## Calculation of import shares
    pi = np.zeros((N, N))  # Import shares (what proportion of total consumption in country n is accounted for by goods from country i).
                           # Here, the exporting country is the first dimension, and the exporting country is the next dimension.
    pi_num = np.zeros((N, N))  # Numerator part of import shares
    Phi = np.zeros((N))  # Denominator part of import shares

    for OR, DE in np.ndindex((N, N)):
        pi_num[OR, DE] = T[OR] * (w[OR] * d[OR, DE]) ** (-theta)
        Phi[DE] += pi_num[OR, DE]

    for OR, DE in np.ndindex((N, N)):
        pi[OR, DE] = pi_num[OR, DE] / Phi[DE]

    # Calculation of price indices
    P = gamma((theta + sigma - 1) / theta) ** (1 / (1 - sigma)) * Phi ** (-1 / theta)

    # Calculation of import values
    X = np.zeros((N, N))
    for OR, DE in np.ndindex((N, N)):
        X[OR, DE] = pi[OR, DE] * Xn[DE]

    ## Calculation of excess labor demand
    L_S = L
    L_D = np.zeros((N))
    for OR, DE in np.ndindex((N, N)):
        L_D[OR] += X[OR, DE] / w[OR]

    Z = L_D - L_S

    # Wage update and normalization
    w_new = w * (1 + psi * (Z / L))
    wgdp = np.sum(w_new * L)
    w_new = w_new / wgdp

    return w_new, Z, X, P

def solve_eqm(theta, sigma, N, L, T, d, psi, tol, maxiter):
    # Initialization of wages and excess labor demand
    Z = np.ones((N))
    w = np.ones((N))
    iter = 1

    # Exit the loop if excess labor demand is sufficiently small
    while max(np.abs(Z)) > tol and iter < maxiter:
        iter += 1
        w_old = np.copy(w)
        w, Z, _, _ = updatewage(w, theta=4, sigma=3, N=3, L_S=L, T=T, d=d, psi=0.1)
        if iter % 10 == 0:
            print(iter)

    if iter == maxiter:
        print("Not done")
        return w, Z, P, X
    else:
        w, Z, X, P = updatewage(w, theta=4, sigma=3, N=3, L_S=L, T=T, d=d, psi=0.1)
        return w, Z, P, X


Using this function, we calculate the equilibrium from the parameters.

In [6]:
N = 3  # Number of countries
theta = 4  # Parameter for trade elasticity
sigma = 3  # Parameter for elasticity of substitution
T = np.array([1., 1., 1.])  # Parameter for technology
L = np.array([1, 1.5, 1.5])  # Population
d = np.ones((N, N)) * 1.5  # Trade costs
for OR in np.ndindex((N)):
    d[OR, OR] = 1  # Domestic trade costs are set to 1
# Solve for equilibrium
w, _, P, X = solve_eqm(theta, sigma, N, L, T, d, psi=0.1, tol=0.00001, maxiter=1000)
print(w)


10
20
[0.26061868 0.24646044 0.24646044]


We think this as an initial equlibirum, and calculate counterfactual equilibrim. We first assume parameters are known and resolve the
equilibrium under the alternative parameters. We change the trade costs from $\boldsymbol{\tau} = 1.5$ to $\boldsymbol{\tau}' = 1.2$  (we fix the intranational trade costs).

In [7]:
d_new = np.ones((N,N)) * 1.2 # New trade costs.
for OR in np.ndindex((N)):
  d_new[OR,OR] = 1 # Keep the domestic trade costs to 1.
print(d_new)

dhat = d_new / d
print(dhat)

[[1.  1.2 1.2]
 [1.2 1.  1.2]
 [1.2 1.2 1. ]]
[[1.  0.8 0.8]
 [0.8 1.  0.8]
 [0.8 0.8 1. ]]


When we resolve the equilibrium and compare the real wage (welfare)...

In [8]:
w_cf,_,P_cf,X_cf = solve_eqm(theta,sigma,N,L,T,d_new,psi=0.1,tol=0.00001,maxiter=1000)

U = w / P
U_cf = w_cf / P_cf
print(U_cf/U)

10
[1.10939542 1.08093162 1.08093162]


# Introducing Exact Hat Algebra

Using Exact Hat Algebra, we confirm the counterfactual equilibrium from trade liberalizations. Specifically, we compare the result of exact hat
algebra and the solving approach.

For Exact hat algebra, calculate total absorption and import share.

In [9]:
# Calculate total absorption
Xn = np.sum(X,axis=0) # np.sum(X,axis=0) means that to sum over X in 1st dimension.

# Calculate import share
pi = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  pi[OR,DE] = X[OR,DE] / Xn[DE]
print(X)
print(Xn)
print(pi)

[[0.17444744 0.04308621 0.04308621]
 [0.04308556 0.2727316  0.05387291]
 [0.04308556 0.05387291 0.2727316 ]]
[0.26061855 0.36969072 0.36969072]
[[0.66935926 0.11654664 0.11654664]
 [0.16532037 0.7377291  0.14572427]
 [0.16532037 0.14572427 0.7377291 ]]


Temporaliry set $\boldsymbol{\hat{w}} = 1$ and check whether Exact hat algebra equilibrium holds.

In [10]:
what = np.ones((N))
print(what)

[1. 1. 1.]


Since total absorption changes, we calculate the counterfactual absorptions:
$$ X_{i}' = \hat{w}_{i} X_{i}. $$

In [11]:
Xn1 = what * Xn
print(Xn1)

[0.26061855 0.36969072 0.36969072]


Calculate $\boldsymbol{\hat{\pi}}$ from  $\boldsymbol{\hat{w}}$:
$$ \hat{\pi}_{ni} = \frac{(\hat{w}_i \hat{d}_{ni})^{-\theta}}{ \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} }. $$
Calculate price index changes:
$$ \hat{P}_n = \left( \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} \right)^{-1/\theta}. $$

In [12]:
pihat = np.zeros((N,N))
pihat_num = np.zeros((N,N))
pihat_den = np.zeros((N))

for OR,DE in np.ndindex((N,N)):
  pihat_num[OR,DE] = (what[OR] * dhat[OR,DE] )**(-theta)
  pihat_den[DE] += pi[OR,DE] * pihat_num[OR,DE] # pihat_den[DE] = pihat_den[DE] + pihat_num[OR,DE]

for OR,DE in np.ndindex((N,N)):
  pihat[OR,DE] = pihat_num[OR,DE] / pihat_den[DE]

Phat = pihat_den ** (-1/theta)

# Confirm that the new import share sums up to one.
print(pihat)
print(pihat * pi)
print(np.sum(pihat*pi,axis=0))

[[0.67723715 1.77165261 1.77165261]
 [1.65341102 0.72566891 1.77165261]
 [1.65341102 1.77165261 0.72566891]]
[[0.45331496 0.20648016 0.20648016]
 [0.27334252 0.53534707 0.25817278]
 [0.27334252 0.25817278 0.53534707]]
[1. 1. 1.]


Calculate new export values.
$$ X'_{ni} = \pi_{ni} \hat{\pi}_{ni} X'_{i} $$

In [13]:
X1 = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X1[OR,DE] = pi[OR,DE] * pihat[OR,DE] * Xn1[DE]


Under this temporal wages, $\hat{\boldsymbol{w}}$, confirm is the labor maket is cleared. The labor market clearing implies:

$$  L^S_{i} = \frac{\sum_{n=1}^N X_{ni}'}{w'_i}. $$

Mulitplying both sides with $w_i'$ and express everything in hat yields:
$$ \hat{w}_i w_i L_{i} = \sum_{n=1}^N X_{ni}' $$
We calcula the differences between labor supply and labor demand as excess labor demand.

In [14]:
wL_D = np.zeros((N))
wL_S = what * Xn

for OR,DE in np.ndindex((N,N)):
  wL_D[OR] += X1[OR,DE]

Z = (wL_D - wL_S) / what
print(Z)

[ 0.01019133 -0.00509567 -0.00509567]


Update the $\boldsymbol{\hat{w}}$ from the excess labor demand:
$$ \hat{w}_{new,i} = \hat{w}_i * \left(1 + \psi * \frac{Z_i}{X'_{i}}\right). $$

In [15]:
psi = 0.1 # The parameter governs the convergence speed.

what_new = what * (1 + psi * (Z / Xn1) )

Compare the new $\boldsymbol{\hat{w}}$ and the old $\boldsymbol{\hat{w}}$. The wage changes should be higher if there is an excess labor demand.

In [16]:
print(Z)
print(what_new - what)

[ 0.01019133 -0.00509567 -0.00509567]
[ 0.00391044 -0.00137836 -0.00137836]


The changes in equlibrium wages need normalization. We normalizes so that the equilbirium global GDP is constant.

In [17]:
wgdp1 = np.sum(what_new * Xn1)
wgdp = np.sum(Xn)
print(wgdp1)
what_new = what_new / wgdp1 * wgdp
print(what_new)

1.0
[1.00391044 0.99862164 0.99862164]


# Functionalize Exact Hat Algebra

So far, the flow can be summarized as follows:
1. Set the parameters
2. Based on the assumed wage changes, calculate the changes in import shares and trade values
3. Using the new trade values, check the equilibrium of the labor market
4. Update the assumed wage changes using the excess labor demand

We will summarize steps 2 to 4 into a **function**.

In [18]:
def updatewagehat(what, dhat, theta, N, X, psi):
    # Calculate total demand
    Xn = np.sum(X, axis=0)
    Xn1 = what * Xn

    # Calculate import shares
    for OR, DE in np.ndindex((N, N)):
        pi[OR, DE] = X[OR, DE] / Xn[DE]

    # Calculate changes in import shares
    pihat = np.zeros((N, N))
    pihat_num = np.zeros((N, N))
    pihat_den = np.zeros((N))

    for OR, DE in np.ndindex((N, N)):
        pihat_num[OR, DE] = (what[OR] * dhat[OR, DE]) ** (-theta)
        pihat_den[DE] += pi[OR, DE] * pihat_num[OR, DE]

    for OR, DE in np.ndindex((N, N)):
        pihat[OR, DE] = pihat_num[OR, DE] / pihat_den[DE]

    # Calculate changes in price indices
    Phat = pihat_den ** (-1 / theta)

    # Calculate new trade values
    X1 = np.zeros((N, N))
    for OR, DE in np.ndindex((N, N)):
        X1[OR, DE] = pi[OR, DE] * pihat[OR, DE] * Xn1[DE]

    # Calculate labor market equilibrium in the new equilibrium
    wL_D = np.zeros((N))
    wL_S = what * Xn

    for OR, DE in np.ndindex((N, N)):
        wL_D[OR] += X1[OR, DE]

    Z = (wL_D - wL_S) / what

    # Update and normalize wages
    what_new = what * (1 + psi * (Z / Xn1))
    wgdp1 = np.sum(Xn1)
    wgdp = np.sum(Xn)
    what_new = what_new * wgdp / wgdp1

    return what_new, Z, Phat, X1

Check whether this function is consistent with the previous result (without function).

In [19]:
what_newfunc,_,_,_ = updatewagehat(what,dhat=dhat,theta=theta,N=N,X=X,psi=psi)
print(what_newfunc)
print(what_new)

[1.00391044 0.99862164 0.99862164]
[1.00391044 0.99862164 0.99862164]


Use while-loop and update $\hat{\boldsymbol{w}}$ until the equilibrium condition is satisfied.


In [20]:
tol = 0.000001
iter = 1

Z = np.ones((N))
what = np.ones((N))
while max(np.abs(Z)) > tol and iter < 100000:
  iter += 1
  what_old = np.copy(what)
  what,Z,Phat,_ = updatewagehat(what,dhat=dhat,theta=theta,N=N,X=X,psi=0.1)
  if iter % 10 == 0:
    print(iter)
    print(Z)
    print(what)

  if max(np.abs(Z)) < tol:
    print("Done!")
    print(iter)
    print(Z)
    print(what)


10
[ 8.19915634e-05 -4.14682847e-05 -4.14682847e-05]
[1.00852779 0.99699398 0.99699398]
Done!
18
[ 6.89840595e-07 -3.48928383e-07 -3.48928383e-07]
[1.00856618 0.99698058 0.99698058]


Check if this exact hat-algebra result is consistent with the resolved equilibrim:

$$ \hat{U}_i = \hat{w}_i / \hat{P}_i. $$

In [21]:
Uhat = what / Phat
Uhat_rs = U_cf/U
what_rs = w_cf/w

print(Uhat)
print(Uhat_rs)
print(what)
print(what_rs)

[1.10939608 1.0809314  1.0809314 ]
[1.10939542 1.08093162 1.08093162]
[1.00856618 0.99698058 0.99698058]
[1.00856551 0.99698081 0.99698081]
